In [9]:
import pandas as pd
import numpy as np

from IPython.core.display import display_html

from bokeh.embed import file_html
from bokeh.models import ColumnDataSource, Patches, HoverTool, TapTool, Plot, Range1d, Callback, Slider
from bokeh.palettes import Blues9
from bokeh.plotting import vplot
from bokeh.resources import Resources
from jinja2 import Template


from constants import PLOT_FORMATS, DARK_GRAY

### Get the data assembled and colored

The following code to get the data assembled is explained in more detail in the notebook "Example pretty static map with data & hover"

In [10]:
map_data = pd.read_hdf('data/province_map_data.hdf', 'df')
map_data.sort('alpha', inplace=True)
map_data = map_data.set_index('alpha')
data = pd.read_csv('data/sample_data_by_year.csv', index_col='alpha')

In [11]:
# Make the column names ints not strings for handling
columns = list(data.columns)
years = list(range(int(columns[0]), int(columns[-1])))
data = data.rename(columns=dict(zip(columns, years)))
data[1990][0:5]

alpha
AH    94.083611
BJ    60.270488
CQ    17.200208
FJ    62.326373
GD    29.758603
Name: 1990, dtype: float64

In [12]:
def color_data(data, columns_to_colorify, data_min=None, data_max=None, palette=Blues9):
    # data - the data frame which you are adding colored values to
    # columns_to_colorify - a list of strings which select the columns
    
    if data_min is None:
        num_only = data[columns_to_colorify]
        global_min = num_only.min().min()
        data_min = np.floor(global_min)

    if data_max is None:
        num_only = data[columns_to_colorify]
        global_max = num_only.max().max()
        data_max = np.ceil(global_max)
    
    data_range = data_max - data_min
    bin_factor = data_range / len(palette)
    colored = pd.DataFrame()
    
    def _get_color(value, palette):
        index = int(value / bin_factor)
        return palette[index - 1]

    for column_name in columns_to_colorify:
        colored[column_name] = data[column_name].apply(_get_color, args=([palette]))
    return colored

colored_data = color_data(data, years)
colored_data.head()

colored_data[1990][0:5]

alpha
AH    #deebf7
BJ    #6baed6
CQ    #08306b
FJ    #6baed6
GD    #08519c
Name: 1990, dtype: object

In [13]:
map_data['xs'].head()

alpha
AH    (116.8389978365808, 116.84773115386969, 116.90...
BJ    (117.07273034133209, 117.0876131526249, 117.09...
CQ    (109.3359932795662, 109.57308475126024, 109.59...
FJ    [117.43482506600012, 117.44778172300019, 117.4...
GD    [110.44263756600006, 110.44752037900005, 110.4...
Name: xs, dtype: object

In [14]:
sources = {}

xs = map_data['xs']
xs.name = 'xs'
ys = map_data['ys']
ys.name = 'ys'

for year in years:
    series = colored_data[year]
    series.name = 'color'
    new_df = pd.concat([series, xs, ys], axis=1)
    sources['_' + str(year)] = ColumnDataSource(new_df)
    
dictionary_of_sources = dict(zip([x for x in years], ['_%s' % x for x in years]))
js_source_array = str(dictionary_of_sources).replace("'", "")

## Map Code

In [15]:
def build_interactive_map(data_frame, plot_width=600, x_range=[70, 140], y_range=[10, 60], title=""):
    aspect_ratio = (x_range[1] - x_range[0]) / (y_range[1] - y_range[0])
    plot_height = int(plot_width / aspect_ratio)
    x_range = Range1d(x_range[0], x_range[1])
    y_range = Range1d(y_range[0], y_range[1])
    plot = Plot(
        x_range=x_range, 
        y_range=y_range, 
        title=title, 
        plot_width=plot_width, 
        plot_height=plot_height, 
        **PLOT_FORMATS)

    source = ColumnDataSource(data_frame)

    tooltips = "<span class='tooltip-text year'>@active_year</span>"                   
    tooltips += "<span class='tooltip-text country'>@name_en</span>"               
    tooltips += "<span class='tooltip-text value'>@active_value</span>"                                
    plot.add_tools(HoverTool(tooltips=tooltips))

    countries = Patches(
        xs='xs', 
        ys='ys',
        fill_color='color',
        line_color=DARK_GRAY,
        line_width=0.5,
    )    

    renderer_source = sources['_%s' % years[0]]
    plot.add_glyph(renderer_source, countries)

    code = """
    var year = slider.get('value'),
        sources = %s,
        new_source_data = sources[year].get('data');
    renderer_source.set('data', new_source_data);
    renderer_source.trigger('change');
    """ % js_source_array
    
    callback = Callback(args=sources, code=code)
    slider = Slider(start=years[0], end=years[-1], value=1, step=1, title="Year", callback=callback)
    callback.args['slider'] = slider
    callback.args['renderer_source'] = renderer_source

    layout = vplot(plot, slider)
    return layout

## Construct map

In [16]:
PLOT_WIDTH = 900
TITLE = 'Random data'
layout = build_interactive_map(colored_data, plot_width=PLOT_WIDTH)

with open('interactive_map_template.jinja', 'r') as f:
    template = Template(f.read())

resources = Resources(mode='inline')

# Update these to change the text
template_variables = {
    'title': TITLE,
    'narrative': 'Some explanatory text.',
    'bokeh_min_js': resources.js_raw[0],
}

### DON'T USE THIS TECHNIQUE - RESULTS IN A MASSIVE FILE - DUE TO MAP DETAIL!

# Use inline resources, render the html and open
#html = file_html(layout, resources, TITLE, template=template, template_variables=template_variables)


# Uncomment the next two lines if you'd like to save the file
#with open('interactive_map_with_slider_1.html', 'w') as f:
#    f.write(html)

#display_html(html, raw=True)